In [1]:
# preprocess_data_200.py
import xarray as xr

def preprocess_geopotential(filepath):
    ds = xr.open_dataset(filepath)
    z200 = ds['z'].sel(pressure_level=200).mean(dim='valid_time')
    anomalies = ds['z'].sel(pressure_level=200) - z200
    anomalies.to_netcdf('z200_anomalies.nc')

if __name__ == "__main__":
    preprocess_geopotential('india_pressure_level200_2016_2021.nc')  # Replace with your actual file if needed


In [2]:
# eof_clustering_200.py
import xarray as xr
import numpy as np
from eofs.xarray import Eof
from sklearn.cluster import KMeans

def run_eof_clustering():
    data = xr.open_dataset('z200_anomalies.nc')['z']
    if 'valid_time' in data.dims:
        data = data.rename({'valid_time': 'time'})

    solver = Eof(data)
    pcs = solver.pcs(npcs=7, pcscaling=1)
    np.save('pcs_200.npy', pcs.values)

    variance = solver.varianceFraction().values[:7]
    np.save('explained_variance_200.npy', variance)

    pcs_np = pcs.values
    pcs_norm = (pcs_np - pcs_np.mean(axis=0)) / pcs_np.std(axis=0)

    kmeans = KMeans(n_clusters=4, n_init=10, random_state=42).fit(pcs_norm)
    labels = kmeans.labels_

    regimes_ds = xr.Dataset({'regime': (['time'], labels)}, coords={'time': data['time']})
    regimes_ds.to_netcdf('eof_weather_regimes_200.nc')

    print("✅ EOF clustering (200 hPa) completed and saved.")

if __name__ == "__main__":
    run_eof_clustering()


✅ EOF clustering (200 hPa) completed and saved.


In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import os
import pandas as pd
from scipy.interpolate import interp1d

def ensure_time(ds):
    if 'valid_time' in ds.dims:
        ds = ds.rename({'valid_time': 'time'})
    return ds

def plot_regime_frequency(ds):
    labels, counts = np.unique(ds['regime'].values, return_counts=True)
    plt.figure(figsize=(7,5))
    sns.barplot(x=labels, y=counts, palette="deep")
    plt.title("Weather Regime Frequency (2016–2021)", fontsize=14)
    plt.xlabel("Regime", fontsize=12)
    plt.ylabel("Days", fontsize=12)
    plt.tight_layout()
    plt.savefig('plots_200/regime_frequency.png', dpi=300)
    plt.close()

def plot_spatial_composites(ds):
    for r in np.unique(ds['regime'].values):
        with xr.open_dataset('z200_anomalies.nc') as anomalies:
            anomalies = ensure_time(anomalies)
            z = anomalies['z']
            z_regime = z.where(ds['regime'] == r).mean(dim='time')

            plt.figure(figsize=(8,6))
            ax = plt.axes(projection=ccrs.PlateCarree())
            z_regime.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                                   levels=np.arange(-200, 200, 20),
                                   cmap='RdBu_r', extend='both', add_colorbar=True)
            ax.coastlines()
            ax.set_title(f"Regime {r}: Mean Z200 Anomaly", fontsize=14)
            plt.tight_layout()
            plt.savefig(f'plots_200/regime_{r}_z200_composite.png', dpi=300)
            plt.close()

def plot_pc_timeseries(pcs, nao_index, enso_df):
    with xr.open_dataset('z200_anomalies.nc') as anomalies:
        anomalies = ensure_time(anomalies)
        times = anomalies['time'].values

    explained_variance = np.load('explained_variance_200.npy')
    pc1_var = explained_variance[0] * 100

    enso_df['date'] = pd.to_datetime(enso_df[['Year', 'Month']].assign(day=15))
    enso_df = enso_df.set_index('date').loc["2016":"2021"]

    plt.figure(figsize=(10,5))
    plt.plot(times, pcs[:, 0], label='PC1 (EOF1)', linewidth=1.5)
    plt.plot(nao_index['time'].values, nao_index['nao'].values, label='NAO Index', linewidth=1.5)
    plt.plot(enso_df.index, enso_df['Anomaly'], label='ENSO Index', linewidth=1.5)
    plt.legend()
    plt.title(f"PC1 vs NAO and ENSO Index (2016–2021)\nPC1 Explained Variance = {pc1_var:.2f}%", fontsize=14)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Index Value", fontsize=12)
    plt.tight_layout()
    plt.savefig('plots_200/pc1_vs_enso_nao.png', dpi=300)
    plt.close()

def plot_seasonal_cycle(ds):
    ds = ensure_time(ds)
    times = ds['time'].to_index()
    months = times.month
    regimes = ds['regime'].values
    plt.figure(figsize=(10,5))
    for r in np.unique(regimes):
        monthly_counts = [np.sum((months==m) & (regimes==r)) for m in range(1,13)]
        plt.plot(range(1,13), monthly_counts, label=f'Regime {r}')
    plt.xticks(range(1,13), ['Jan','Feb','Mar','Apr','May','Jun',
                             'Jul','Aug','Sep','Oct','Nov','Dec'])
    plt.legend()
    plt.title("Seasonal Cycle of Regime Occurrence", fontsize=14)
    plt.xlabel("Month", fontsize=12)
    plt.ylabel("Days", fontsize=12)
    plt.tight_layout()
    plt.savefig('plots_200/seasonal_cycle.png', dpi=300)
    plt.close()

def plot_pc_index_correlation(pcs, index_array, label, file_name, time_values=None, index_time=None):
    if time_values is not None and index_array.shape[0] != pcs.shape[0]:
        # Convert pcs to DataFrame with datetime index
        df = pd.DataFrame(pcs, columns=[f"PC{i+1}" for i in range(pcs.shape[1])])
        df['date'] = pd.to_datetime(time_values)
        df = df.set_index('date')
        pcs_monthly = df.resample('M').mean()

        # Create aligned DataFrame
        index_df = pd.Series(index_array, index=index_time)
        index_df = index_df.loc[pcs_monthly.index]

        # Now calculate correlations
        correlations = [np.corrcoef(pcs_monthly.iloc[:, i], index_df)[0, 1] for i in range(pcs.shape[1])]
    else:
        correlations = [np.corrcoef(pcs[:, i], index_array)[0, 1] for i in range(pcs.shape[1])]

    plt.figure(figsize=(8,5))
    sns.barplot(x=[f'PC{i+1}' for i in range(7)], y=correlations, palette="deep")
    plt.ylim(-1, 1)
    plt.title(f"Correlation between EOF PCs and {label} (2016–2021)", fontsize=14)
    plt.ylabel("Pearson Correlation", fontsize=12)
    plt.xlabel("Principal Components", fontsize=12)
    plt.tight_layout()
    plt.savefig(f'plots_200/{file_name}.png', dpi=300)
    plt.close()


def plot_spatial_composites_new(ds):
    pcs = np.load('pcs_200.npy')
    explained_variance = np.load('explained_variance_200.npy')
    labels = ds['regime'].values
    explained_variance = explained_variance / explained_variance.sum()

    regime_variance_map = {}
    for r in np.unique(labels):
        regime_pcs = pcs[labels == r]
        pc_var = np.var(regime_pcs, axis=0)
        pc_var_normalized = pc_var / pc_var.sum()
        regime_variance = np.sum(pc_var_normalized * explained_variance)
        regime_variance_map[r] = regime_variance

    for r in np.unique(ds['regime'].values):
        with xr.open_dataset('z200_anomalies.nc') as anomalies:
            anomalies = ensure_time(anomalies)
            z = anomalies['z']
            z_regime = z.where(ds['regime'] == r).mean(dim='time')

            plt.figure(figsize=(8, 6))
            ax = plt.axes(projection=ccrs.PlateCarree())
            z_regime.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                                   levels=np.arange(-200, 200, 20),
                                   cmap='RdBu_r', extend='both', add_colorbar=True)
            ax.coastlines()
            variance_str = f"{regime_variance_map[r]*100:.2f}%"
            ax.set_title(f"Regime {r}: Mean Z200 Anomaly\n(Weighted EOF Variance ≈ {variance_str})", fontsize=13)
            plt.tight_layout()
            plt.savefig(f'plots_200_rand/regime_{r}_z200_composite.png', dpi=300)
            plt.close()

def main():
    os.makedirs("plots_200", exist_ok=True)
    os.makedirs("plots_200_rand", exist_ok=True)

    with xr.open_dataset('eof_weather_regimes_200.nc') as ds:
        ds = ensure_time(ds)
        pcs = np.load('pcs_200.npy')

        # Load NAO index
        with xr.open_dataset('nao_index.nc') as nao_index:
            # Load ENSO index CSV
            enso_df = pd.read_csv('Enso_Monthwise_Index.csv')
            enso_df['date'] = pd.to_datetime(enso_df[['Year', 'Month']].assign(day=15))
            # Convert to month-end to match resampled PC dates
            enso_df['date'] = pd.to_datetime(enso_df[['Year', 'Month']].assign(day=1)) + pd.offsets.MonthEnd(0)
            enso_df = enso_df.set_index('date').loc["2016-01-31":"2021-12-31"]
            enso_series = enso_df['Anomaly'].values

            # Interpolate NAO to match pcs time
            anomalies = xr.open_dataset('z200_anomalies.nc')
            anomalies = ensure_time(anomalies)
            time_values = anomalies['time'].values
            f_nao = interp1d(nao_index['time'].values.astype(np.int64),
                             nao_index['nao'].values, kind='linear', fill_value="extrapolate")
            nao_interp = f_nao(time_values.astype(np.int64))

            plot_regime_frequency(ds)
            plot_spatial_composites(ds)
            plot_pc_timeseries(pcs, nao_index, enso_df)
            plot_pc_index_correlation(pcs, nao_interp, "NAO Index", "pc_nao_correlation")
            plot_pc_index_correlation(
                pcs,
                enso_series,
                "ENSO Index",
                "pc_enso_correlation",
                time_values=time_values,
                index_time=enso_df.index
            )

            plot_seasonal_cycle(ds)
            plot_spatial_composites_new(ds)

if __name__ == "__main__":
    main()


C:\Users\medhr\AppData\Local\Temp\ipykernel_13420\3824886403.py:18: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=labels, y=counts, palette="deep")
C:\Users\medhr\AppData\Local\Temp\ipykernel_13420\3824886403.py:104: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=[f'PC{i+1}' for i in range(7)], y=correlations, palette="deep")
C:\Users\medhr\AppData\Local\Temp\ipykernel_13420\3824886403.py:92: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pcs_monthly = df.resample('M').mean()
C:\Users\medhr\AppData\Local\Temp\ipykernel_13420\3824886403.py:104: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be remove

In [ ]:
def plot_extreme_risk_ratio(power_data, regimes):
    thresholds = np.percentile(power_data, 5)  # define extreme event threshold
    rr = []
    for r in np.unique(regimes):
        regime_days = power_data[regimes == r]
        p_wr = np.sum(regime_days < thresholds) / len(regime_days)
        p_clim = np.sum(power_data < thresholds) / len(power_data)
        ratio = p_wr / p_clim if p_clim > 0 else np.nan
        rr.append(ratio)
    plt.figure(figsize=(8,6))
    sns.barplot(x=np.unique(regimes), y=rr)
    plt.title("Extreme Energy Shortfall Risk Ratio by Regime")
    plt.xlabel("Regime")
    plt.ylabel("Risk Ratio")
    plt.savefig("plots/extreme_risk_ratio.png", dpi=300)
    plt.close()

def plot_pc_scatter_trajectories(pcs, regimes):
    plt.figure(figsize=(8,6))
    scatter = plt.scatter(pcs[:,0], pcs[:,1], c=regimes, cmap='tab10', s=10)
    plt.colorbar(scatter, label='Regime')
    plt.title("Regime Transitions in PC1-PC2 Phase Space")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.savefig("plots/pc_phase_space.png", dpi=300)
    plt.close()

def plot_teleconnection_correlation(pcs, teleconnection_indices):
    pc_array = pcs.values
    indices_array = teleconnection_indices.values
    combined = np.hstack([pc_array, indices_array])
    corr_matrix = np.corrcoef(combined.T)

    plt.figure(figsize=(10,8))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", 
                xticklabels=['PC1','PC2','PC3','NAO','IOD','MJO'],
                yticklabels=['PC1','PC2','PC3','NAO','IOD','MJO'])
    plt.title("Correlation Matrix: PCs vs Teleconnections")
    plt.savefig("plots/teleconnection_correlation.png", dpi=300)
    plt.close()

def plot_composite_anomalies(ds, variable_name, levels=None, cmap="RdBu_r"):
    if levels is None:
        levels = np.linspace(-2, 2, 21)
    for r in np.unique(ds['regime'].values):
        subset = ds[variable_name].where(ds['regime'] == r).mean(dim='time')
        plt.figure(figsize=(8,6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        subset.plot.contourf(ax=ax, transform=ccrs.PlateCarree(),
                             levels=levels, cmap=cmap, extend='both')
        ax.coastlines()
        ax.set_title(f"Regime {r}: {variable_name} Anomaly Composite")
        plt.savefig(f"plots/{variable_name}_regime_{r}_composite.png", dpi=300)
        plt.close()

# plot_composite_anomalies(ds, 'u_component_of_wind')
# plot_composite_anomalies(ds, 'v_component_of_wind')
# plot_composite_anomalies(ds, 'temperature')
# plot_composite_anomalies(ds, 'relative_humidity')



In [ ]:
def plot_summary_panel(ds):
    fig, axes = plt.subplots(2,2, figsize=(12,10))
    
    # A: Regime frequency
    labels, counts = np.unique(ds['regime'].values, return_counts=True)
    axes[0,0].bar(labels, counts)
    axes[0,0].set_title("Regime Frequency")
    
    # B: Seasonal cycle
    times = ds['time'].to_index()
    months = [t.month for t in times]
    regimes = ds['regime'].values
    for r in np.unique(regimes):
        monthly_counts = [np.sum((np.array(months)==m) & (regimes==r)) for m in range(1,13)]
        axes[0,1].plot(range(1,13), monthly_counts, label=f'Regime {r}')
    axes[0,1].legend()
    axes[0,1].set_title("Seasonal Cycle")
    axes[0,1].set_xticks(range(1,13))
    axes[0,1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    
    # C: Phase space
    pcs = ds['pcs'].values if 'pcs' in ds else np.random.randn(len(ds['time']), 2)  # dummy example
    scatter = axes[1,0].scatter(pcs[:,0], pcs[:,1], c=ds['regime'].values, cmap='tab10', s=10)
    fig.colorbar(scatter, ax=axes[1,0])
    axes[1,0].set_title("PC1-PC2 Phase Space")
    
    # D: Co-occurrence matrix
    # wind_regimes & solar_regimes must be defined
    # Dummy example
    matrix = np.random.randint(0,100,(4,4))
    sns.heatmap(matrix, annot=True, fmt="0.0f", cmap="viridis", ax=axes[1,1])
    axes[1,1].set_title("Wind vs Solar Co-occurrence")
    
    plt.tight_layout()
    plt.savefig("plots/summary_panel.png", dpi=300)
    plt.close()


In [ ]:
def main():
    ds = xr.open_dataset('eof_weather_regimes.nc')
    pcs = xr.open_dataset('z500_anomalies.nc')['z']  # or your PCs dataset
    tele = xr.open_dataset('teleconnections.nc')  # must be prepared
    
    power_data = np.load('daily_renewable_power.npy')  # must be prepared
    regimes = ds['regime'].values
    wind_regimes = regimes  # example
    solar_regimes = regimes  # example

    plot_extreme_risk_ratio(power_data, regimes)
    plot_pc_scatter_trajectories(pcs, regimes)
    plot_teleconnection_correlation(pcs, tele)
    plot_composite_anomalies(ds, 'u_component_of_wind')
    plot_composite_anomalies(ds, 'v_component_of_wind')
    plot_composite_anomalies(ds, 'temperature')
    plot_composite_anomalies(ds, 'relative_humidity')
    plot_summary_panel(ds)

if __name__ == "__main__":
    main()
